In [1]:
import bokeh
import pandas as pd

from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import CARTODBPOSITRON, CARTODBPOSITRON_RETINA

In [11]:
# # data = pd.read_csv('/Users/sam/Downloads/seattle-code-violation-cases/code-violation-cases.csv')
# # data.head()

# # data.describe()

# # data.isnull().sum()

# # data['Case Type'].unique()

# data['Date Case Created'].min()

# data['Date Case Created'].max()

# data.loc[data['Case Type'] == 'TENANT RELOCATION ORDINANCE']

# data.loc[data['Case Type'] == 'TENANT RELOCATION ORDINANCE', 'Date Case Created'].sort_values()

In [12]:
# data = pd.read_csv('/Users/sam/Downloads/SPD_Reports.csv')
# data

In [13]:
# data = pd.read_csv('/Users/sam/Downloads/Seattle_Police_Department_911_Incident_Response.csv')
# data

In [24]:
data = pd.read_csv('/Users/sam/Downloads/kc_house_data.csv')
# data

In [25]:
data['year'] = data['date'].apply(lambda x: int(str(x)[:4]))

In [18]:
data['year'].unique()

array([2014, 2015])

In [20]:
# test = data.loc[data['year'] == 2014, ['price', 'lat', 'long']]

In [26]:
def coords_to_plot(lat, lon):
    
    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return (x, y)

In [106]:
p = figure(x_range=(-13630000, -13610000), y_range=(6038000, 6046000),
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(CARTODBPOSITRON_RETINA)

colors_dict = {2014: '#008000', 2015: '#FF0000'}

for year in data['year'].unique():
    
    coords = data.loc[data['year'] == year, ['price', 'lat', 'long']]
    coords['merc'] = coords.apply(lambda x: coords_to_plot(x['lat'], x['long']), axis=1).values
    
    for coord in coords.index[:500]:
        
        x, y, price = coords.loc[coord]['merc'][0], coords.loc[coord]['merc'][1], coords.loc[coord]['price']

        p.circle(x = x,
                 y = y,
                 size=price/100000,
                 fill_alpha=.2,
                 line_color = colors_dict[year],
                 fill_color = colors_dict[year]
                )

output_notebook()
show(p)

Loading BokehJS ...

In [29]:
# Make sample data

coords = data.loc[data['year'] == 2014, ['price', 'lat', 'long']]

In [37]:
def normalize(coords_price):
    
    cord_min = coords_price.min()
    cord_max = coords_price.max()
    
    def scale(cord):
        return (cord - cord_min)/(cord_max-cord_min)
    
    return coords_price.apply(lambda x: scale(x))
        
coords['normed'] = normalize(coords['price'])

In [65]:
coords = coords[:2000]

In [69]:
def json(row):
    return {'lat': row['lat'], 'long': row['long'], 'value': round(row['normed'], 6)} 
        
data = list(coords.apply(lambda x: json(x), axis=1).values)

In [71]:
import json

In [72]:
with open('data.json', 'w') as fp:
    json.dump(data, fp)

In [50]:
# # No lat/long
# data = pd.read_csv('/Users/sam/Downloads/seattle-use-of-force/use-of-force.csv')
# data

In [51]:
# # No lat/long
# data = pd.read_csv('/Users/sam/Downloads/seattle-rent-and-income-restricted-housing/rent-and-income-restricted-housing.csv')
# data